In [13]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix

# Definicja funkcji generującej dane
def generate_vet_eye_data(num_records=1000):
    """Generuje syntetyczny DataFrame dla Vet-Eye CRM."""
    segment_options = ["klinika", "mobilny", "szpital"]
    segment_weights = [0.6, 0.2, 0.2] # Przykładowe wagi dla segmentów
    data = []
    for i in range(1, num_records + 1):
        client_id = f"CL-{i:04d}"
        segment = random.choices(segment_options, weights=segment_weights, k=1)[0]

        if segment == "mobilny":
            clinic_size = random.randint(1, 2)
            devices_owned = random.randint(1, 2)
        elif segment == "klinika":
            clinic_size = random.randint(2, 15)
            devices_owned = random.randint(1, 4)
        else:  # szpital
            clinic_size = random.randint(10, 50)
            devices_owned = random.randint(2, 8)

        last_purchase_days_ago = random.randint(7, 730)
        purchase_count = random.randint(1, 30)
        avg_purchase_value = round(random.uniform(500.0, 15000.0), 2)

        tu2_active = random.choices([0, 1], weights=[0.3, 0.7], k=1)[0]

        if tu2_active == 1:
            tu2_sessions_last_30d = random.randint(0, 200)
            ai_usage_ratio = round(random.uniform(0.0, 1.0), 2)
        else:
            tu2_sessions_last_30d = 0
            ai_usage_ratio = 0.0

        last_contact_days_ago = random.randint(1, 365)
        open_rate = round(random.uniform(0.05, 0.9), 2)
        click_rate = round(random.uniform(0.0, open_rate * 0.5), 2)
        support_tickets_last_6m = random.randint(0, 10)

        buy_chance = 0.1
        if 30 < last_purchase_days_ago < 180: buy_chance += 0.1
        if tu2_sessions_last_30d > 50: buy_chance += 0.1
        if avg_purchase_value > 5000: buy_chance += 0.05
        buy_label = 1 if random.random() < buy_chance else 0

        churn_chance = 0.05
        if support_tickets_last_6m > 5: churn_chance += 0.1
        if tu2_active == 1 and tu2_sessions_last_30d < 10: churn_chance += 0.05
        if last_contact_days_ago > 180: churn_chance += 0.1
        if not tu2_active: churn_chance += 0.1
        churn_label = 1 if random.random() < min(churn_chance, 0.8) else 0

        if buy_label == 1 and churn_label == 1:
            if random.random() < 0.7: churn_label = 0
        if tu2_active == 0 and last_purchase_days_ago > 180 :
            if random.random() < 0.3 : churn_label = 1

        data.append([
            client_id, segment, clinic_size, devices_owned,
            last_purchase_days_ago, purchase_count, avg_purchase_value,
            tu2_active, tu2_sessions_last_30d, ai_usage_ratio,
            last_contact_days_ago, open_rate, click_rate,
            support_tickets_last_6m, buy_label, churn_label
        ])

    columns = [
        "client_id", "segment", "clinic_size", "devices_owned",
        "last_purchase_days_ago", "purchase_count", "avg_purchase_value",
        "tu2_active", "tu2_sessions_last_30d", "ai_usage_ratio",
        "last_contact_days_ago", "open_rate", "click_rate",
        "support_tickets_last_6m", "buy_label", "churn_label"
    ]
    return pd.DataFrame(data, columns=columns)

print("Komórka 1: Biblioteki zaimportowane, funkcja generująca dane zdefiniowana.")

Komórka 1: Biblioteki zaimportowane, funkcja generująca dane zdefiniowana.


In [14]:
# --- Konfiguracja Globalna ---
DATA_FILE_NAME = "vet_eye_crm_data_1000_PL.csv"
MODEL_UPSELL_FILE = "model_upsell.json"
MODEL_CHURN_FILE = "model_churn.json"
RANDOM_STATE = 42
TEST_SIZE = 0.2

print("Komórka 2: Generowanie danych syntetycznych (1000 rekordów)...")
df_global = generate_vet_eye_data(num_records=1000)

df_global.to_csv(DATA_FILE_NAME, index=False, encoding='utf-8')
print(f"Dane wygenerowane i zapisane do '{DATA_FILE_NAME}' (w środowisku Colab).")
print(f"Liczba rekordów: {len(df_global)}")

X_source_global = df_global.drop(['client_id', 'buy_label', 'churn_label'], axis=1)
y_upsell_source_global = df_global['buy_label']
y_churn_source_global = df_global['churn_label']

categorical_features_global = ['segment']

preprocessor_global = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_global)
    ],
    remainder='passthrough'
)
print("Dane przygotowane i zmienne globalne zdefiniowane.")

Komórka 2: Generowanie danych syntetycznych (1000 rekordów)...
Dane wygenerowane i zapisane do 'vet_eye_crm_data_1000_PL.csv' (w środowisku Colab).
Liczba rekordów: 1000
Dane przygotowane i zmienne globalne zdefiniowane.


In [15]:
from sklearn.base import clone

def train_evaluate_model_generic(model_type_name, X_full, y_full, preprocessor_to_use, cat_features_list, model_output_filename):
    print(f"\n--- Rozpoczęcie Pracy nad Modelem: {model_type_name} ---")

    X_train, X_test, y_train, y_test = train_test_split(
        X_full, y_full, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y_full
    )

    print(f"Rozmiar zbioru treningowego: {X_train.shape[0]}, Rozmiar zbioru testowego: {X_test.shape[0]}")

    current_preprocessor = clone(preprocessor_to_use)

    X_train_processed = current_preprocessor.fit_transform(X_train)
    X_test_processed = current_preprocessor.transform(X_test)

    # Próba pobrania nazw cech po transformacji
    # Ta część może być wrażliwa na wersje sklearn, staramy się być elastyczni
    processed_feature_names = None
    try:
        processed_feature_names = list(current_preprocessor.get_feature_names_out())
    except AttributeError:
        try:
            # Starsza metoda dla OneHotEncoder w ColumnTransformer
            ohe_feature_names = list(current_preprocessor.named_transformers_['onehot'].get_feature_names_out(cat_features_list))
            # Nazwy cech, które przeszły przez 'passthrough'
            num_feature_names_indices = [i for i, t in enumerate(current_preprocessor.transformers_) if t[0] == 'remainder' and t[1] == 'passthrough'][0][2]
            remainder_feature_names = [X_full.columns[i] for i in num_feature_names_indices]
            processed_feature_names = ohe_feature_names + remainder_feature_names
        except Exception as e:
            print(f"Ostrzeżenie: Nie udało się automatycznie pobrać nazw cech po transformacji: {e}")
            print("Model XGBoost zostanie wytrenowany bez jawnie zdefiniowanych nazw cech.")
            # W takim przypadku XGBoost sam sobie poradzi, ale może wyświetlić ostrzeżenie, jeśli nazwy będą potrzebne później
            # np. przy SHAP values. Dla samego treningu i predykcji nie jest to krytyczne.

    if processed_feature_names and len(processed_feature_names) != X_train_processed.shape[1]:
        print(f"Ostrzeżenie: Niezgodność liczby nazw cech ({len(processed_feature_names)}) z liczbą kolumn ({X_train_processed.shape[1]}). Użycie domyślnych nazw.")
        processed_feature_names = None


    print(f"Liczba cech po przetworzeniu dla {model_type_name}: {X_train_processed.shape[1]}")
    if processed_feature_names:
        print(f"Przykładowe przetworzone nazwy cech: {processed_feature_names[:5]}...")
    else:
        print("Model będzie trenowany bez jawnie przekazanych nazw przetworzonych cech.")


    scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1) if np.sum(y_train == 1) > 0 else 1
    print(f"{model_type_name} - scale_pos_weight: {scale_pos_weight:.2f}")

    model = XGBClassifier(
        objective='binary:logistic',
        n_estimators=150,
        max_depth=5,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        use_label_encoder=False, # To jest ważne
        scale_pos_weight=scale_pos_weight,
        eval_metric=['logloss', 'auc'], # POPRAWKA: eval_metric jest parametrem KONSTRUKTORA
        early_stopping_rounds=20 # POPRAWKA: early_stopping_rounds jest parametrem KONSTRUKTORA
    )

    print(f"Trening modelu {model_type_name}...")
    model.fit(
        X_train_processed, y_train,
        eval_set=[(X_test_processed, y_test)], # eval_set jest przekazywany do fit
        verbose=False
    )
    print("Trening zakończony.")

    model.save_model(model_output_filename)
    print(f"Model {model_type_name} zapisany do pliku: {model_output_filename}")

    print(f"\nEwaluacja modelu {model_type_name} na zbiorze testowym (używając modelu z 'best_iteration'):")
    y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    y_pred_class = model.predict(X_test_processed)

    # Sprawdzenie, czy atrybut best_iteration istnieje
    if hasattr(model, 'best_iteration_') and model.best_iteration_ is not None: # W Scikit-learn API jest to best_iteration_
         print(f"  Najlepsza iteracja (0-indexed): {model.best_iteration_}")
    elif hasattr(model, 'best_iteration') and model.best_iteration is not None: # Dla pewności, jeśli API się zmieniło
         print(f"  Najlepsza iteracja (0-indexed): {model.best_iteration}")
    else:
        print("  Wczesne zatrzymanie mogło się nie aktywować lub atrybut best_iteration nie jest dostępny w tej wersji/konfiguracji.")

    print(f"  AUC: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(f"  F1-Score: {f1_score(y_test, y_pred_class):.4f}")
    print(f"  Precyzja: {precision_score(y_test, y_pred_class, zero_division=0):.4f}")
    print(f"  Czułość (Recall): {recall_score(y_test, y_pred_class, zero_division=0):.4f}")
    print("  Macierz Pomyłek:")
    cm = confusion_matrix(y_test, y_pred_class)
    print(cm)

    print(f"--- Koniec Pracy nad Modelem: {model_type_name} ---")
    return model

print("Komórka 3: Funkcja do treningu i ewaluacji modelu POPRAWIONA (eval_metric i early_stopping_rounds w konstruktorze XGBClassifier).")

Komórka 3: Funkcja do treningu i ewaluacji modelu POPRAWIONA (eval_metric i early_stopping_rounds w konstruktorze XGBClassifier).


In [16]:
# Trening i ewaluacja modelu Upsell
model_upsell = train_evaluate_model_generic(
    model_type_name="Upsell (Buy Label)",
    X_full=X_source_global.copy(),
    y_full=y_upsell_source_global,
    preprocessor_to_use=preprocessor_global,
    cat_features_list=categorical_features_global,
    model_output_filename=MODEL_UPSELL_FILE
)


--- Rozpoczęcie Pracy nad Modelem: Upsell (Buy Label) ---
Rozmiar zbioru treningowego: 800, Rozmiar zbioru testowego: 200
Liczba cech po przetworzeniu dla Upsell (Buy Label): 15
Przykładowe przetworzone nazwy cech: ['onehot__segment_klinika', 'onehot__segment_mobilny', 'onehot__segment_szpital', 'remainder__clinic_size', 'remainder__devices_owned']...
Upsell (Buy Label) - scale_pos_weight: 4.30
Trening modelu Upsell (Buy Label)...
Trening zakończony.
Model Upsell (Buy Label) zapisany do pliku: model_upsell.json

Ewaluacja modelu Upsell (Buy Label) na zbiorze testowym (używając modelu z 'best_iteration'):
  Najlepsza iteracja (0-indexed): 1
  AUC: 0.5736
  F1-Score: 0.2833
  Precyzja: 0.2073
  Czułość (Recall): 0.4474
  Macierz Pomyłek:
[[97 65]
 [21 17]]
--- Koniec Pracy nad Modelem: Upsell (Buy Label) ---


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:08:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [17]:
# Trening i ewaluacja modelu Churn
model_churn = train_evaluate_model_generic(
    model_type_name="Churn Label",
    X_full=X_source_global.copy(),
    y_full=y_churn_source_global,
    preprocessor_to_use=preprocessor_global,
    cat_features_list=categorical_features_global,
    model_output_filename=MODEL_CHURN_FILE
)


--- Rozpoczęcie Pracy nad Modelem: Churn Label ---
Rozmiar zbioru treningowego: 800, Rozmiar zbioru testowego: 200
Liczba cech po przetworzeniu dla Churn Label: 15
Przykładowe przetworzone nazwy cech: ['onehot__segment_klinika', 'onehot__segment_mobilny', 'onehot__segment_szpital', 'remainder__clinic_size', 'remainder__devices_owned']...
Churn Label - scale_pos_weight: 3.68
Trening modelu Churn Label...
Trening zakończony.
Model Churn Label zapisany do pliku: model_churn.json

Ewaluacja modelu Churn Label na zbiorze testowym (używając modelu z 'best_iteration'):
  Najlepsza iteracja (0-indexed): 6
  AUC: 0.7571
  F1-Score: 0.4516
  Precyzja: 0.4200
  Czułość (Recall): 0.4884
  Macierz Pomyłek:
[[128  29]
 [ 22  21]]
--- Koniec Pracy nad Modelem: Churn Label ---


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:08:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [18]:
# --- Instrukcja Pobierania Plików z Colab ---
print("\n\n--- Zakończono Cały Skrypt ---")
print(f"Plik danych został zapisany jako '{DATA_FILE_NAME}'.")
print(f"Modele zostały zapisane jako '{MODEL_UPSELL_FILE}' i '{MODEL_CHURN_FILE}'.")
print("Wszystkie te pliki znajdują się w bieżącym środowisku sesji Colab.")
print("Aby je pobrać na swój komputer:")
print("1. Po lewej stronie w interfejsie Colab kliknij ikonę folderu (Pliki).")
print("2. Na liście plików (może być konieczne odświeżenie) znajdź:")
print(f"   - {DATA_FILE_NAME}")
print(f"   - {MODEL_UPSELL_FILE}")
print(f"   - {MODEL_CHURN_FILE}")
print("3. Kliknij na trzy kropki obok nazwy każdego pliku i wybierz opcję 'Pobierz'.")



--- Zakończono Cały Skrypt ---
Plik danych został zapisany jako 'vet_eye_crm_data_1000_PL.csv'.
Modele zostały zapisane jako 'model_upsell.json' i 'model_churn.json'.
Wszystkie te pliki znajdują się w bieżącym środowisku sesji Colab.
Aby je pobrać na swój komputer:
1. Po lewej stronie w interfejsie Colab kliknij ikonę folderu (Pliki).
2. Na liście plików (może być konieczne odświeżenie) znajdź:
   - vet_eye_crm_data_1000_PL.csv
   - model_upsell.json
   - model_churn.json
3. Kliknij na trzy kropki obok nazwy każdego pliku i wybierz opcję 'Pobierz'.
